In [ ]:
!pip install schedule


In [ ]:
import schedule
import time

def enviar_correos():


  import pandas as pd
  import smtplib
  from email.mime.text import MIMEText
  from email.mime.multipart import MIMEMultipart
  from email.mime.image import MIMEImage

  # Importación del archivo Excel data1
  df = pd.read_excel("/content/data 1.xlsx")

  # Filtrar los pedidos cancelados
  canceled_orders = df[df["Status"] == "Canceled"]

  # Se obtiene las direcciones de correo electrónico de los pedidos cancelados
  email_addresses = canceled_orders["Email"].tolist()

  # Configuracion del servidor de correo electrónico de Outlook
  outlook_server = "smtp.office365.com"
  outlook_port = 587
  outlook_email = "user@Falabella.com"
  outlook_password = "contraseña"

  # Configuración el servidor de correo electrónico
  server = smtplib.SMTP(outlook_server, outlook_port)
  server.starttls()

  # Inicia sesión en el servidor de Outlook
  server.login(outlook_email, outlook_password)

  # Itera a través de los pedidos cancelados y envía el correo a cada destinatario
  for index, row in canceled_orders.iterrows():
      # Crea un mensaje de correo electrónico para cada destinatario
      message = MIMEMultipart()
      message["From"] = outlook_email
      message["Subject"] = "SKU Status: Canceled"

      # Cuerpo del mensaje en formato HTML con la firma al final
      body = f"""
      <html>
      <body>
          <p>Estimado <strong>{row['sellerName']}</strong>,</p>
          <p>Su pedido asociado al SKU <strong>{row['Sku']}</strong> está <strong>{row['Status']}</strong>.</p>
          <p>Gracias por su atención.</p>
          <p>---</p>
          <p><img src="cid:firma_image"></p>
      </body>
      </html>
      """
      message.attach(MIMEText(body, "html"))

      # Adjunta la imagen de la firma
      with open("/content/Firma.png", "rb") as image_file:
          firma_image = MIMEImage(image_file.read())
          firma_image.add_header("Content-ID", "<firma_image>")
          message.attach(firma_image)

      # Envía el correo electrónico
      message["To"] = row["Email"]
      server.sendmail(outlook_email, row["Email"], message.as_string())

  # Cierra la conexión con el servidor de correo electrónico
  server.quit()

# Programar la ejecución del código todos los días a las 13:40
schedule.every(2).minutes.do(enviar_correos)

# Ejecutar el programa de planificación en un bucle infinito
try:
    while True:
        schedule.run_pending()
        time.sleep(1)  # Esperar 6 segundos antes de verificar si hay tareas programadas
except KeyboardInterrupt:
    print("Programa detenido por el usuario")